# Word Embeddings para clasificación binaria

In [1]:
import spacy
import sklearn
import pandas as pd
import numpy as np
import string

2021-11-10 17:37:31.335133: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-11-10 17:37:31.335221: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Descargamos el modelo pre-entrenado de Spacy para español, `es_core_news_lg`.
Este modelo contiene embeddings tanto para palabras como para lexemas en español, a través de la clase Vocab.
Siguiendo el ejemplo en inglés, usamos los vectores de los lexemas.
No obstante, los tokens devueltos por el objeto `nlp` tienen una propiedad `vector`.


In [7]:
nlp = spacy.load('es_core_news_lg')

In [15]:
def clean_text(text):
    '''Removing punctuation, lowercasing and removing extra whitespace'''
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation+'¿¡'))
    text = text.replace('\n', ' ')
    text = ' '.join(text.split())
    return text

In [12]:
clean_string("Qué onda?? ¿Cómo has estado?")

'qué onda cómo has estado'

In [55]:
def embed(tokens, nlp):
    """
    Returns the centroid of embeddings of the given tokens
    Out-of-vocabulary and stopwords are ignored
    If no tokens are valid, zero vector is returned
    """
    lexemes = (nlp.vocab[token] for token in tokens)
    
    vectors = np.asarray([
        lexeme.vector
        for lexeme in lexemes
        if lexeme.has_vector
        and not lexeme.is_stop
        and len(lexeme.text) > 1
    ])

    if len(vectors) > 0:
        centroid = vectors.mean(axis=0)
    else:
        width = nlp.meta['vectors']['width']  # fastText Wikipedia Spanish
        centroid = np.zeros(width)

    return centroid

In [84]:
def embedText(text, nlp):
    tokens = nlp(text)
    vectors = np.asarray([
        tok.vector
        for tok in tokens
        if tok.has_vector
        and not tok.is_stop
        and len(tok.text) > 1
    ])
    if len(vectors) > 0:
        centroid = vectors.mean(axis=0)
    else:
        width = nlp.meta['vectors']['width']  # fastText Wikipedia Spanish
        centroid = np.zeros(width)

    return centroid

In [85]:
example0 = "Jamás había estado tan enojado por haberme olvidado del cumpleaños de mi novia."
example1 = "Los perros son criaturas hermosas, yo pienso que hacen una muy buena compañía."
example2 = "No todos los bares de café son buenos. La semana pasada fui a Quentin y estuvo terrible!"

In [89]:
embedText(example0, nlp)

array([ 0.8814755 ,  0.26515839, -1.1960168 ,  0.81276554,  0.14816168,
       -0.620755  , -2.1197166 ,  1.2053455 , -0.3045357 ,  1.0702027 ,
       -0.29849663,  0.27251   ,  0.75110286, -0.97496504, -1.0333383 ,
       -0.09135284,  0.39116168,  0.26902798, -0.90455   , -1.4397825 ,
       -0.32542336, -0.2597683 , -0.26377735, -0.09344831,  0.14625494,
       -0.4337617 , -1.2746733 , -0.56135005, -0.2610416 ,  0.8261278 ,
       -0.964745  ,  0.15281157,  0.81725836, -1.2116791 , -1.429565  ,
       -0.33164504, -1.2847348 ,  0.83878833,  1.0524    , -0.34363332,
       -0.3970833 ,  1.4156533 ,  1.237555  , -0.21057999,  0.614205  ,
       -0.61506987, -0.27823162, -1.1454316 , -1.0071689 ,  0.06216697,
        0.43626752, -0.01546101, -0.8033333 , -0.24647065,  0.42837167,
       -0.25064597,  0.3403615 ,  0.4721782 , -0.09022667,  2.09517   ,
        0.38766   ,  0.8042307 , -0.77815   ,  1.2508299 ,  0.28359166,
       -0.20560698, -1.8716683 , -0.14955033,  0.0772867 ,  1.07

In [58]:
tokens0

['jamás',
 'había',
 'estado',
 'tan',
 'enojado',
 'por',
 'haberme',
 'olvidado',
 'del',
 'cumpleaños',
 'de',
 'mi',
 'novia']

# Clasificación no supervisada con spaCy

In [60]:
def get_label_embeddings(label_names, nlp):
    '''
    Dada una lista de nombres de clase, regresa los embeddings 
    correspondientes a cada una.
    El nombre de la clase puede tener varias palabras.
    '''
    
    label_embeddings = np.asarray([
        embed(name.split(), nlp)
        for name in label_names
    ])
    
    return label_embeddings

In [91]:
label_names = ["excelente", "decepcionante"]
label_embeddings = get_label_embeddings(label_names, nlp)

In [108]:
from sklearn.neighbors import NearestNeighbors

nb = NearestNeighbors(n_neighbors=2)
nb.fit(label_embeddings)  # label centroids are training data for clusters

NearestNeighbors(n_neighbors=2)

In [109]:
# "Jamás había estado tan enojado por haberme olvidado del cumpleaños de mi novia"
closest_label = nb.kneighbors([centroid0], return_distance=False)[0, 0]
label_names[closest_label]

'decepcionante'